In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(master='local', appName='TransformacionesyAcciones')

In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [5]:
rdd1.collect()

[1, 2, 3]

In [6]:
sc

<SparkContext master=local appName=TransformacionesyAcciones>

# Carga de paises

In [7]:
path = '/home/cesarppz/Documents/master/code/python/spark/recursos_curso/curso-apache-spark-platzi/files/'

In [25]:
equiposOlimpicosRDD = sc.textFile(path+'paises.csv').map(lambda x : x.split(','))

In [26]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

## Contar cuantos paises hay

In [29]:
equiposOlimpicosRDD.map(lambda x : x[2]).distinct().count()

231

In [43]:
equiposOlimpicosRDD.filter(lambda x : 'AUT' in x).collect()

[['1', '30. Februar', 'AUT'],
 ['71', 'Austria', 'AUT'],
 ['72', 'Austria-1', 'AUT'],
 ['73', 'Austria-2', 'AUT'],
 ['143', 'Breslau', 'AUT'],
 ['145', 'Brigantia', 'AUT'],
 ['293', 'Donar III', 'AUT'],
 ['334', 'Evita VI', 'AUT'],
 ['651', 'May-Be 1960', 'AUT'],
 ['837', '"R.-V. Germania; Leitmeritz"', 'AUT'],
 ['1007', 'Surprise', 'AUT']]

In [55]:
equiposOlimpicosRDD.map(lambda x : (x[2],x[:2])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [38]:
equiposOlimpicosRDD.countApprox(20)

1185

# TRabajar con dos RDD

In [48]:
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [49]:
deportistaOlimpicoRDD.count()

135572

In [56]:
deportistaOlimpicoRDD.top(3)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967'],
 ['99998', 'Robert John Bob Renney', '1', '21', '178', '90', '66']]

In [68]:
 deportistaOlimpicoRDD.map(lambda x : [x[-1], x[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x : [x[0], x[2]])) \
    .takeSample(False, 5, 25)

[('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('154', (['39161', 'Angel Merdzhanov Gavrilov', '1', '24', '0', '0'], 'BUL')),
 ('1084',
  (['62843', 'Olha Vasylivna Korobka', '2', '18', '181', '167'], 'UKR')),
 ('678', (['97550', 'Puntsagiin Skhbat', '1', '24', '174', '82'], 'MGL')),
 ('1096', (['106789', 'Hugo Scherzer', '1', '43', '0', '0'], 'USA'))]

In [73]:
resultado = sc.textFile(path+'resultados.csv').map(lambda x : x.split(','))
resultado.top(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99999', 'NA', '50604', '51', '568'],
 ['99998', 'NA', '50603', '47', '36']]